In [ ]:
import warnings
import numpy as np

warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

from thermoengine.const import units
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))

from tcg_slb.phasediagram.equilibrate import EquilibratePD, EquilibratePDGrid, PDEquilibrateGridDiagnostics
from tcg_slb.base import *
from thermoengine.equilibrate import PhaseLibrary
import thermoengine as thermo
import json
import pickle

with open("sammon.json") as json_file:
    comps = json.load(json_file)

comp = thermo.OxideWtComp(**comps["LC"])

In [ ]:

db = thermo.model.Database(database='Berman')

phases = db.get_phases([
        "Qz",
        "Grt",  
        "Jd","Aeg","CaTs","cEn",
        "Di","En","Fs",
        "Fsp",
        "Rt","Ilm",
])

phase_library = PhaseLibrary(phases)

reload = True


filename = os.path.join('output', 'equilibrate_berman-sammon-LC.pickle')

if reload:
    pfile = open(filename, 'rb')
    equilgridB = pickle.load(pfile)
    pfile.close()
else:
    Tmin = 573
    Tmax = 1273
    nT = 30
    Trange = np.linspace(Tmin,Tmax,nT)
    Pmin = 0
    Pmax = 2.5
    nP = 30
    Prange = np.linspace(Pmin,Pmax,nP)
    print(Prange)
    equilgridB = EquilibratePDGrid()
    equilgridB.solve(phase_library, ['T', 'p'], Trange, Prange,**{'comp':comp})

save = True

if not reload and save:
        pfile = open(filename, 'wb')
        pickle.dump(equilgridB, pfile)
        pfile.close()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

def decorate(pdrgd):
    def new_setup_axes(self,axi):
        axi.set_xlim([773,1273])
        return axi

    #replace the display with newdisplay
    pdrgd.setup_axes = new_setup_axes
    
    #return the modified student 
    return pdrgd

equildiagB = decorate(PDEquilibrateGridDiagnostics)(phase_library, equilgridB)
equildiagB.plot_rho()
equildiagB.plot_phases()



In [ ]:
db = thermo.model.Database(database='Stixrude')

phases = db.get_phases([
        "Qz",
        "Grt",  
        "Cpx",
        "Opx",
        "Ky",
        "Fsp"
        #"Rt","Ilm",
        #"Mc"
])
phase_library = PhaseLibrary(phases)

reload = False


import pickle

filename = os.path.join('output', 'equilibrate_stixrude-sammon-LC.pickle')

if reload:
    pfile = open(filename, 'rb')
    equilgrid = pickle.load(pfile)
    pfile.close()
else:
    Tmin = 773
    Tmax = 1273
    nT = 60
    Trange = np.linspace(Tmin,Tmax,nT)
    Pmin = 0
    Pmax = 2.5
    nP = 60
    Prange = np.linspace(Pmin,Pmax,nP)
    print(Prange)
    equilgrid = EquilibratePDGrid()
    equilgrid.solve(phase_library, ['T', 'p'], Trange, Prange,**{'comp':comp})

save = True

if not reload and save:
        pfile = open(filename, 'wb')
        pickle.dump(equilgrid, pfile)
        pfile.close()

In [ ]:
equildiag = PDEquilibrateGridDiagnostics(phase_library, equilgrid)
equildiag.plot_rho()
equildiag.plot_phases()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
#from __future__ import annotations # Enable Python 4 type hints in Python 3
from thermoengine.equilibrate import PhaseLibrary, GibbsMinimizer, System
import thermoengine as thermo
from thermoengine.const import units
from thermoengine.core import UnorderedList
import os
import contextlib
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import math
import matplotlib.colors as mcolors
import thermoengine as thermo

Tmin = 500
Tmax = 1200
nT = 2

Pmin = 0.5*units.GPA
Pmax = 2.5*units.GPA
nP = 2

T = np.linspace(Tmin,Tmax,nT)
P = np.linspace(Pmin,Pmax,nP)

phase_symbols_berman = [
    "Qz", "Coe",
    "Grt",  
    'Di','Jd','cEn', #'Cpx'
    'En', 'Fs',
    "Ky",
    "Ab", "An",
    "Rt",
    "Mc"
]

oxides={
        "SiO2"  : 68.0,
        "TiO2"  : 0.66,
        "Al2O3" : 15.1, 
        "FeO"   : 5.21,
        "MnO"   : 0.10, 
        "MgO"   : 2.29, 
        "CaO"   : 2.75, 
        "Na2O"  : 2.63, 
        "K2O"   : 3.11,
        "P2O5"  : 0.17,
    }

Tmin = np.amin(T)
Tmax = np.amax(T)
Pmin = np.amin(P)
Pmax = np.amax(P)

oxide_comp = thermo.OxideWtComp(**oxides)

db = thermo.model.Database(database='Berman')
phases = db.get_phases(phase_symbols_berman)
phase_library = PhaseLibrary(phases)

system = System(
    T=800,
    P=0.25*units.GPA,
    comp=oxide_comp,
    options={'grid_spacing':0.2},
    phase_library=phase_library,
    affinity_thresh=1000.
)